In [10]:
from api import rag
from dotenv import load_dotenv
import os

In [11]:
load_dotenv()
API_KEY = os.getenv('GOOGLE_API_KEY')
os.environ["http_proxy"] = "http://127.0.0.1:2081"

In [12]:
result = rag.get_suggestions('where can I find best pizza')
result

['{"restaurant_name": "Slices Pizza", "review": "Pizza is pretty good, prices are reasonable, portions are decent, and they\'re open late on the weekends. Location is pretty convenient too - especially coming from one of the local bars. I think Frank\'s has better pizza, but they close much earlier.", "location": "Philadelphia, PA, 4249 Ridge Ave"}', '{"restaurant_name": "Fiamme Pizza", "review": "We had their pizza for a long time at the Rillito farmer\'s market and finally they have a brick and table place. This is hands down one of the best pizzas I have had and among the top 3 (Florence included). It is thin crust so it you are looking for Chicago style, you will be disappointed. Their caprese pizza with pine nuts is amazing. They have won international competitions and it shows.", "location": "Tucson, AZ, 4706 E Sunrise Dr"}', '{"restaurant_name": "Top Tomato Bar & Pizza", "review": "Okay pizza. My co-workers rave about it but I found their pizza to be average.  They do have a hug

'{"greeting": "Hey there!  I\'m excited to help you find the best pizza in town. Let\'s explore some delicious options!", "suggestions": [{"restaurant_name": "Fiamme Pizza", "note": "Reviewers consistently praise Fiamme Pizza as one of the best, even comparing it favorably to pizzerias in Florence, Italy.  They highlight their thin crust and amazing caprese pizza.", "conclusion": "If you appreciate high-quality, award-winning pizza, Fiamme Pizza is an excellent choice."}, {"restaurant_name": "Fresco Pizzeria", "note": "This pizzeria receives rave reviews, particularly for its gluten-free and dairy-free options, which are described as phenomenal even by non-restricted eaters.  The wide topping selection at a great price is also a plus.", "conclusion": "Fresco Pizzeria is highly recommended, especially if you need gluten-free or dairy-free options, but even otherwise, it\'s considered exceptional."}, {"restaurant_name": "La Bella Pizzeria & Ristorante Italiano", "note": "Considered the b

In [18]:
rag.get_summary('Fiamme Pizza')


'{"restaurant_name": "Fiamme Pizza", "must_try_dishes": ["Carne pizza", "Margherita pizza", "Figo pizza", "Caprese salad", "Caprese pizza with pine nuts"], "highlights": "Amazing, authentic Neapolitan-style pizza with a wonderfully thin and crispy crust. Fresh, high-quality ingredients.  Friendly and passionate staff.  Excellent lunch specials. Many reviewers claim it\'s the best pizza in Tucson, some even comparing it favorably to Pizzeria Bianco.", "notes": "Slow service is a recurring complaint. Limited seating and can be very busy, leading to long wait times.  Some mention inconsistencies with certain dishes (e.g., burnt breadsticks, garlic bread). Beverage options are considered limited.  Interior decor could be improved.", "conclusion": "Fiamme Pizza receives overwhelmingly positive feedback for its exceptional pizza, using fresh, high-quality ingredients and boasting an authentic Neapolitan style. However, the small space and slow service are significant drawbacks. Despite this,

In [10]:
rag.restaurant_qna('Fiamme Pizza', 'what is the vibe of the restaurant?')

'{"restaurant_name": "Fiamme Pizza", "answer": "Fiamme Pizza has a small, intimate atmosphere, described as being reminiscent of a narrow restaurant in Boston\'s North End.  Many reviewers note the limited seating and busy environment, often recommending off-peak hours to avoid long waits.  The restaurant is immaculate and the staff is consistently praised for being friendly and welcoming.  The overall vibe is described as casual and laid-back, with a focus on high-quality, authentic Italian pizza and a warm, inviting atmosphere."}'

In [6]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings, StorageContext, load_index_from_storage, QueryBundle
from llama_index.vector_stores.elasticsearch import ElasticsearchStore
from llama_index.llms.gemini import Gemini

/home/parichehr/PycharmProjects/NLP/llm-restaurant-recommender/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
Settings.embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")
Settings.chunk_size = 2048
llm = Gemini(model_name='models/gemini-1.5-flash', api_key=API_KEY)

vector_store = ElasticsearchStore(
    index_name='restaurant_reviews',
    vector_field='review_vector',
    text_field='review',
    es_url='http://localhost:9200/'
)

storage_context = StorageContext.from_defaults(persist_dir='../notebooks/storage',
                                               vector_store=vector_store)
index = load_index_from_storage(storage_context=storage_context, index_id='vector_index')
query_engine = index.as_query_engine(llm, similarity_top_k=10)

# Query
query = "suggest best pizza spot"
bundle = QueryBundle(
    query_str=query,
    embedding=Settings.embed_model.get_query_embedding(query=query)
)
response = query_engine.query(bundle)
print(response.response)

/tmp/ipykernel_25136/2318627713.py:3: DeprecationWarning: Call to deprecated class Gemini. (Should use `llama-index-llms-google-genai` instead, using Google's latest unified SDK. See: https://docs.llamaindex.ai/en/stable/examples/llm/google_genai/)
  llm = Gemini(model_name='models/gemini-1.5-flash', api_key=API_KEY)


Loading llama_index.core.storage.kvstore.simple_kvstore from ../notebooks/storage/docstore.json.
Loading llama_index.core.storage.kvstore.simple_kvstore from ../notebooks/storage/index_store.json.
Several restaurants received top ratings.  One reviewer called a particular Indianapolis pizza place their favorite, praising its gourmet pizza with creative, fresh ingredients and flavorful dough. Another reviewer declared a Medford, NJ restaurant the best in the area, highlighting its excellent Margherita and Buffalo chicken pizzas, large slices, and friendly staff.  A Reno, NV establishment was also lauded for its amazing crust, fresh ingredients, and nice staff, with one reviewer calling it the best pizza in town.  Finally, a Santa Barbara, CA spot was described as having well-prepared and tasty pizza, making it a great place for a casual hangout.



In [ ]:
# from huggingface_hub import login
# login(token=HF_TOKEN)

In [ ]:
os.environ["TORCH_USE_CUDA_DSA"]='1'
os.environ["CUDA_LAUNCH_BLOCKING"]='1'

In [ ]:
# llm = HuggingFaceLLM(model_name="distilgpt2",
#                      device_map="cuda:0",
#                      max_new_tokens=100,
#                      )
llm = Gemini(model="models/gemma-3-27b-it", api_key=API_KEY)

query_engine = index.as_query_engine(llm, similarity_top_k=2)
# Query
query = "what is best pizza spot in new york"
bundle = QueryBundle(
    query_str=query,
    embedding=Settings.embed_model.get_query_embedding(query=query)
)
response = query_engine.query(bundle)
print(response.response)

In [ ]:
import google.generativeai as genai

genai.configure(api_key=API_KEY)

model = genai.GenerativeModel("gemma-3-27b-it")
response = model.generate_content("What are sardines?")
print(response.text)

In [1]:
from elasticsearch import Elasticsearch
es = Elasticsearch("http://localhost:9200")

In [3]:
es.indices.get(index="*")

ObjectApiResponse({'restaurant_reviews': {'aliases': {}, 'mappings': {'properties': {'metadata': {'properties': {'_node_content': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}, '_node_type': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}, 'doc_id': {'type': 'keyword'}, 'document_id': {'type': 'keyword'}, 'location': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}, 'rating': {'type': 'float'}, 'rating_category': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}, 'ref_doc_id': {'type': 'keyword'}, 'restaurant_name': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}, 'review': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}}}, 'review': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}, 'review_vector': {'type': 'dense_vector', 'dims': 384, 'index': True, 's

In [ ]:
es.indices.delete(index='yelp_restaurant_reviews')